In [1]:
pip install opencv-python easyocr

Note: you may need to restart the kernel to use updated packages.


In [7]:
import torch
torch.cuda.is_available()

True

In [8]:
import cv2
import easyocr
import json
from PIL import Image, ImageDraw

In [9]:
def preprocess_image(image_path):
    # Read the image
    image = cv2.imread(image_path)
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    
    # Use adaptive thresholding to create a binary image
    thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 11, 2)
    
    # Dilate the image to connect individual elements
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    dilate = cv2.dilate(thresh, kernel, iterations=1)
    
    return dilate

In [10]:
def run_model(image_path):
# 이미지 경로


# OpenCV를 사용하여 이미지 읽기

    image = image_path

# EasyOCR을 초기화하고 사용할 언어를 설정
    reader = easyocr.Reader(['en', 'ko'], gpu=True)

# 이미지에서 텍스트 추출
    result = reader.readtext(image)
    
    for (bbox, text, prob) in result:
        print(f'{text}')

    return None

In [16]:
def run_boxModel(image_path):
    import easyocr
    import cv2

# Create an EasyOCR reader
    reader = easyocr.Reader(['en', 'ko'], gpu=True)

    proccessed_iamge = preprocess_image(image_path)

# Detect text and bounding boxes
    results = reader.readtext(proccessed_iamge, width_ths=15, ycenter_ths = 0.5, adjust_contrast = 0.5, add_margin = 0.5)

    image = cv2.imread(image_path)

# Detect text and bounding boxes
    # kernel = np.ones((5,5), np.uint8)
    # # dilate = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
    # dilate = cv2.erode(image, kernel)

    cv2.imwrite('/Users/402-07/YoungMan_projectDE/box/dilate_image.jpg', image)
    flag = False
    text_lists = []
    list = []
    for i, (bbox, text, prob) in enumerate(results):
    # Extract coordinates of the bounding box
        top_left = (int(bbox[0][0]), int(bbox[0][1]))
        bottom_right = (int(bbox[2][0]), int(bbox[2][1]))

    # Crop the bounding box area
        cropped_image = image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
        list.append(cropped_image)
        Image.fromarray(cropped_image).save(f'cropped_text_{i}.png')
            
    for image in list:
        crop_results = reader.recognize(image)
        for (bbox, text, prob) in crop_results:
            # print(text)
            text_lists.append(text)
    print(text_lists)
            
    # Save the cropped image
    # for text in text_lists:
    #     if flag and "판매소계" in text or "판매" in text:
    #         break
    #     if flag:
    #         a.append(text)
    #     if "금액" in text:
    #         flag = True


    
        #print(f"Text: {text}, Confidence: {prob}")
    return None

In [15]:
image_path = '/Users/402-07/YoungMan_projectDE/receipt_list/08.jpg'
run_boxModel(image_path)

['M엔에스컴퍼니', "'2022-04-19", '~', 'Frill', "'5좀 1712", '01076290920)', '01058585f46', "~'--;--=-- :===---------:- ;", '디오트 5증 J1,J2 뜨길', '{장문 010-5858-5646', 'KAKAO trill_', 'insta f+ill_Mholesale', '우리은행 1005-304-300565', 'ECHAT ID +rili_wholesale', '2022-04-19 00:10 (화) 78,UUO   영수(현금', '뚜명   단가   수량   금액', '한스시검정   39,000 1   39,000', '한스j보카베이지   39,000 1   39,000', '만매소계 -   2   2   78,000', 'KCuTk', '잔', '0', '당일말계:   78,000', '연', "'금입금:   78,000", '당   잔', '우리은행 1005-304-300565', '(가기현frill', '반품', '{킬쨌물', '_부구필부둑 -', '끊액입다다', '물들성삼 흰색@L름 제품문 교환물가', '매월 5 일 마감입니다', "'입은 매입일로부터 모개원만 유포맙니", '디오트', '입금처', '#자료는']


In [80]:
def run_redboxModel(image_path):
    import easyocr
    import cv2
# Create an EasyOCR reader
    reader = easyocr.Reader(['en', 'ko'], gpu=True)

    proccessed_image = preprocess_image(image_path)

# Detect text and bounding boxes
    # results = reader.detect(proccessed_image, width_ths=0.7)
    # result = results[0]
    image = cv2.imread(image_path)

    result = reader.readtext(proccessed_image, width_ths = 500, ycenter_ths = 10)
    
    for i, (bbox, text, prob) in enumerate(result):
    # Extract coordinates of the bounding box
        top_left = (int(bbox[0][0]), int(bbox[0][1]))
        bottom_right = (int(bbox[2][0]), int(bbox[2][1]))
        
    # #Unpack the bounding box
    #     (top_left, top_right, bottom_right, bottom_left) = bbox

    # # Convert coordinates to integer
    #     top_left = (int(top_left[0]), int(top_left[1]))
    #     bottom_right = (int(bottom_right[0]), int(bottom_right[1]))

    # Draw a red rectangle on the original image
        cv2.rectangle(image, top_left, bottom_right, (0, 0, 255), 2)  # Red color and 2 px thickness

# Display the image with bounding boxes
    cv2.imwrite('/Users/402-07/YoungMan_projectDE/test_result/image_with_boxes.jpg', image)
    return None

In [226]:
image_path = '/Users/402-08/train/img73.jpg'
run_redboxModel(image_path)

In [222]:
def slice_image(image_path):
    import cv2
    import numpy as np

    # 이미지 로드
    image = cv2.imread(image_path)

    # 이미지 크기 계산
    height, width = image.shape[:2]

    # 이미지를 수평으로 반으로 나눔
    left_side = image[:, :width//2]
    right_side = image[:, width//2:]

    # 왼쪽 부분을 살짝 아래로 내림 (예: 50 픽셀)
    shift_up = 35
    M = np.float32([[1, 0, 0], [0, 1, shift_up]])
    left_shifted = cv2.warpAffine(left_side, M, (left_side.shape[1], left_side.shape[0]))

    # 변환된 이미지를 다시 합침
    shifted_height = left_shifted.shape[0]
    combined = np.zeros((shifted_height, width, 3), dtype=np.uint8)
    combined[:height, :width//2] = left_shifted
    combined[:height, width//2:] = right_side

    # 결과 이미지
    cv2.imwrite('train/redbox_image/sliced_image.jpg', combined)
    return None


In [223]:
image_path = '/Users/402-08/train/img73.jpg'
slice_image(image_path)